## LIBRARIES

In [15]:
import os
from pathlib import Path
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## DATA

In [16]:
os.chdir("D:\\Study\\Deep Learning\\digit-recognizer")

In [17]:
train = pd.read_csv("train.csv")

In [18]:
test = pd.read_csv("test.csv")

In [19]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train.shape, test.shape

((42000, 785), (28000, 784))

## Extracting Input and Target Variable

In [22]:

x=train.drop("label",axis=1)
y=np.array(train['label'])
x.shape,y.shape

((42000, 784), (42000,))

## Train -Test Split -Pytorch

In [60]:
torch_X_train = torch.from_numpy(x.values).type(torch.FloatTensor)/255
torch_y_train = torch.from_numpy(y).type(torch.LongTensor)
myDataset = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
valid_no  = int(0.2 * len(myDataset))
# so divide the data into trainset and testset
trainSet,testSet = torch.utils.data.random_split(myDataset,(len(myDataset)-valid_no,valid_no))
print(f"len of trainSet {len(trainSet)} ,\n len of testSet {len(testSet)}")
batch_size=64
train_loader  = DataLoader(trainSet , batch_size=batch_size ,shuffle=True) 
test_loader  = DataLoader(testSet , batch_size=batch_size ,shuffle=True)

len of trainSet 33600 ,
 len of testSet 8400


## Network

In [66]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 70)
        self.fc3 = nn.Linear(70, 64)
        self.fc4 = nn.Linear(64, 10)
        


        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)

        return x
        
model=Network()
optimizer=optim.Adam(model.parameters(),lr=0.001)
criterion=nn.NLLLoss()

## Train

In [68]:
epochs=8
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader))) 

Epoch: 1/8..  Training Loss: 0.109..  Test Loss: 0.088..  Test Accuracy: 0.974
Epoch: 2/8..  Training Loss: 0.104..  Test Loss: 0.090..  Test Accuracy: 0.973
Epoch: 3/8..  Training Loss: 0.099..  Test Loss: 0.087..  Test Accuracy: 0.975
Epoch: 4/8..  Training Loss: 0.088..  Test Loss: 0.091..  Test Accuracy: 0.973
Epoch: 5/8..  Training Loss: 0.083..  Test Loss: 0.088..  Test Accuracy: 0.976
Epoch: 6/8..  Training Loss: 0.081..  Test Loss: 0.086..  Test Accuracy: 0.974
Epoch: 7/8..  Training Loss: 0.079..  Test Loss: 0.092..  Test Accuracy: 0.974
Epoch: 8/8..  Training Loss: 0.072..  Test Loss: 0.083..  Test Accuracy: 0.976


## Parameters

In [69]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

90764
